In [1]:
#start to 2015-11-02 val set
#2017-3-15 onwards validation

In [2]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
import datetime as dt
import xgboost as xgb

C:\Users\vedpk\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\vedpk\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\vedpk\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [3]:
air_visit =  pd.read_csv('../Input/air_visit_data.csv')
air_store =  pd.read_csv('../Input/air_store_info_proc1.csv')
hpg_store = pd.read_csv('../Input/hpg_store_info_proc1.csv')
air_res = pd.read_csv('../Input/air_reserve.csv')
hpg_res =  pd.read_csv('../Input/hpg_reserve.csv')
store_id =  pd.read_csv('../Input/store_id_relation.csv')
sample_submission =  pd.read_csv('../Input/sample_submission.csv')
date_info =  pd.read_csv('../Input/date_info.csv').rename(columns={'calendar_date':'visit_date'})

In [4]:
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

In [5]:
hpg_res = pd.merge(hpg_res, store_id, how='inner', on=['hpg_store_id'])

In [6]:
air_res['visit_datetime'] = pd.to_datetime(air_res['visit_datetime'])
air_res['visit_datetime'] = air_res['visit_datetime'].dt.date
air_res['reserve_datetime'] = pd.to_datetime(air_res['reserve_datetime'])
air_res['reserve_datetime'] = air_res['reserve_datetime'].dt.date
air_res['reserve_datetime_diff'] = air_res.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
tmp1 = air_res.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
tmp2 = air_res.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
air_res = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

In [7]:
hpg_res['visit_datetime'] = pd.to_datetime(hpg_res['visit_datetime'])
hpg_res['visit_datetime'] = hpg_res['visit_datetime'].dt.date
hpg_res['reserve_datetime'] = pd.to_datetime(hpg_res['reserve_datetime'])
hpg_res['reserve_datetime'] = hpg_res['reserve_datetime'].dt.date
hpg_res['reserve_datetime_diff'] = hpg_res.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
tmp1 = hpg_res.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
tmp2 = hpg_res.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
hpg_res = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

In [8]:
air_visit['visit_date'] = pd.to_datetime(air_visit['visit_date'])
air_visit['dow'] = air_visit['visit_date'].dt.dayofweek
air_visit['year'] = air_visit['visit_date'].dt.year
air_visit['month'] = air_visit['visit_date'].dt.month
air_visit['visit_date'] = air_visit['visit_date'].dt.date

In [9]:
sample_submission['visit_date'] = sample_submission['id'].map(lambda x: str(x).split('_')[2])
sample_submission['air_store_id'] = sample_submission['id'].map(lambda x: '_'.join(x.split('_')[:2]))
sample_submission['visit_date'] = pd.to_datetime(sample_submission['visit_date'])
sample_submission['dow'] = sample_submission['visit_date'].dt.dayofweek
sample_submission['year'] = sample_submission['visit_date'].dt.year
sample_submission['month'] = sample_submission['visit_date'].dt.month
sample_submission['visit_date'] = sample_submission['visit_date'].dt.date

In [10]:
unique_stores = sample_submission['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

In [11]:
air_visit.sort_values('visit_date').iloc[-1]['visit_date']-air_visit.sort_values('visit_date').iloc[0]['visit_date']

datetime.timedelta(477)

In [12]:
air_visit.sort_values('visit_date').iloc[0]['visit_date']+dt.timedelta(days=60)

datetime.date(2016, 3, 1)

In [13]:
air_visit.sort_values('visit_date').iloc[-1]['visit_date']-dt.timedelta(days=38) #38days validation set

datetime.date(2017, 3, 15)

In [14]:
val = air_visit[air_visit['visit_date'] < dt.date(2017, 3, 15)]

In [15]:
val.head()

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


In [16]:
air_visit = air_visit[air_visit['visit_date'] > dt.date(2016, 3, 1)]

In [17]:
#sure it can be compressed...
tmp = air_visit.groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = air_visit.groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = air_visit.groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = air_visit.groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = air_visit.groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

In [18]:
stores = pd.merge(stores, air_store, how='left', on=['air_store_id']) 

stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

In [19]:
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])
date_info['day_of_week'] = lbl.fit_transform(date_info['day_of_week'])
date_info['visit_date'] = date_info['visit_date'].dt.date
train = pd.merge(air_visit, date_info, how='left', on=['visit_date']) 
test = pd.merge(sample_submission, date_info, how='left', on=['visit_date']) 

val = pd.merge(val, date_info, how='left', on=['visit_date'])

In [20]:
train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

val = pd.merge(val, stores, how='left', on=['air_store_id','dow'])

In [21]:
train = pd.merge(train, air_res, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, air_res, how='left', on=['air_store_id','visit_date'])

train = pd.merge(train, hpg_res, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, hpg_res, how='left', on=['air_store_id','visit_date'])

val = pd.merge(val, air_res, how='left', on=['air_store_id','visit_date']) 
val = pd.merge(val, hpg_res, how='left', on=['air_store_id','visit_date'])

In [22]:
train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)
val['id'] = val.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

In [23]:
train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

In [24]:
test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

In [25]:
val['total_reserv_sum'] = val['rv1_x'] + val['rv1_y']
val['total_reserv_mean'] = (val['rv2_x'] + val['rv2_y']) / 2
val['total_reserv_dt_diff_mean'] = (val['rs2_x'] + val['rs2_y']) / 2

In [26]:
#train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
#test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

#val['date_int'] = val['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
val['var_max_lat'] = val['latitude'].max() - val['latitude']
val['var_max_long'] = val['longitude'].max() - val['longitude']

In [27]:
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

val['lon_plus_lat'] = val['longitude'] + val['latitude']

In [28]:
lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

val['air_store_id2'] = lbl.transform(val['air_store_id'])

In [29]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors','prefecture','city','rv1_x','rv1_y','rv2_x','rv2_y']]
train = train.fillna(-1)
test = test.fillna(-1)

val = val.fillna(-1)

In [30]:
#Dropping stores not in test

to_drop = list(set(train['air_store_id']) - set(test['air_store_id']))
to_drop

['air_cb083b4789a8d3a2',
 'air_d63cfa6d6ab78446',
 'air_cf22e368c1a71d53',
 'air_d0a7bd3339c3d12a',
 'air_0ead98dd07e7a82a',
 'air_2703dcb33192b181',
 'air_229d7e508d9f1b5e',
 'air_b2d8bc9c88b85f96']

In [31]:
train = train.where(~(train['air_store_id'].isin(to_drop)))
train = train.dropna(axis=0,subset=['air_store_id'])

In [48]:
val = val.where(~(val['air_store_id'].isin(to_drop)))
val = val.dropna(axis=0,subset=['air_store_id'])

In [33]:
len(train['air_store_id'].unique())

821

In [34]:
#X = train[train['visit_date'] < dt.date(2017, 2, 21)]
X = train

In [35]:
y_train = np.log1p(X['visitors'])
X_train = X[col]

In [36]:
y_val = np.log1p(val['visitors'])
X_val = val[col]

In [37]:
d_train = xgb.DMatrix(X_train,y_train)
d_valid = xgb.DMatrix(X_val, y_val)

In [38]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [49]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      max_depth =10)

In [50]:
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)

In [51]:
model3 = xgb.XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                     colsample_bytree=0.8, max_depth =10)

In [41]:
d_train = xgb.DMatrix(X_train,y_train)
d_valid = xgb.DMatrix(X_val, y_val)

In [42]:
params = {}
params['objective'] = 'reg:linear'
params['booster'] = 'gbtree'
params['eval_metric'] = 'rmse'
params['eta'] = 0.1
params['max_depth'] = 4
params['silent'] = 1
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['tree_method'] = "exact"

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [53]:
model1.fit(train[col], np.log1p(train['visitors'].values))

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=200, presort='auto', random_state=3,
             subsample=0.8, verbose=0, warm_start=False)

In [54]:
model2.fit(train[col], np.log1p(train['visitors'].values))

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=4, p=2,
          weights='uniform')

In [63]:
model3.fit(X_train[col], np.log1p(y_train))

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=200,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=3,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [44]:
model4 = (xgb.train(params,d_train,num_boost_round=2000,evals=watchlist,
                   early_stopping_rounds=100,verbose_eval=10))

[0]	train-rmse:2.20736	valid-rmse:2.20433
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[10]	train-rmse:0.909145	valid-rmse:0.923466
[20]	train-rmse:0.574003	valid-rmse:0.600156
[30]	train-rmse:0.51536	valid-rmse:0.545571
[40]	train-rmse:0.505893	valid-rmse:0.537777
[50]	train-rmse:0.503147	valid-rmse:0.536969
[60]	train-rmse:0.501799	valid-rmse:0.537648
[70]	train-rmse:0.50068	valid-rmse:0.536886
[80]	train-rmse:0.499709	valid-rmse:0.536308
[90]	train-rmse:0.499028	valid-rmse:0.537346
[100]	train-rmse:0.498378	valid-rmse:0.537127
[110]	train-rmse:0.497832	valid-rmse:0.536708
[120]	train-rmse:0.497309	valid-rmse:0.536467
[130]	train-rmse:0.496665	valid-rmse:0.535979
[140]	train-rmse:0.49618	valid-rmse:0.535345
[150]	train-rmse:0.495801	valid-rmse:0.535447
[160]	train-rmse:0.495412	valid-rmse:0.535225
[170]	train-rmse:0.494998	valid-rmse:0.534951
[180]	train-rmse:0.494433	valid-rmse:0.533

In [57]:
preds1 = model1.predict(val[col])

In [58]:
preds2 = model2.predict(val[col])

In [64]:
preds3 = model3.predict(val[col])

In [61]:
pred4 = model4.predict(xgb.DMatrix(val[col]))

In [65]:
print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(val['visitors'].values), preds1))
print('RMSE KNeighborsRegressor: ', RMSLE(np.log1p(val['visitors'].values), preds2))
print('RMSE XGBRegressor: ', RMSLE(np.log1p(val['visitors'].values), preds3))
print('RMSE XGBTrain: ', RMSLE(np.log1p(val['visitors'].values), pred4))

RMSE GradientBoostingRegressor:  0.366855472476
RMSE KNeighborsRegressor:  0.433797705915
RMSE XGBRegressor:  1.62164716171
RMSE XGBTrain:  0.482479349734


In [ ]:
test.head()

In [ ]:
pred4 = model4.predict(test[col])

In [ ]:
#test['visitors'] = (preds2+preds3) / 2 #(preds1+preds2+preds3) / 3
test['visitors'] = (preds1+preds2+preds3+preds4) / 4 
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
sub1 = test[['id','visitors']].copy()
#del train; del data;

In [ ]:
sub_merge['visitors'] = (sub_merge['visitors'])/2
sub_merge[['id', 'visitors']].to_csv('../Submission/submission3.csv', index=False)